In [ ]:
---
sidebar_label: AIMind
---

# ChatAIMind

This notebook provides a quick overview for getting started with AIMind (MindsDB) [chat models](/docs/concepts/#chat-models). 

The [Minds Endpoint](https://docs.mdb.ai/) is a unified, OpenAI-compatible API format to interact with various Minds and LLM providers. This standardization simplifies your access to a wide range of functions like completion, embedding, and image generation. It minimizes the learning curve and accelerates your development process.

It provides a wide selection of [advanced large language models (LLMs)](https://docs.mdb.ai/docs/models) from industry-leading AI providers. 

## Setup

To access MindsDB Endpoint models you'll need to create an MindsDB account, get an API key, and install the `openai` package.

### Credentials

Head to https://mdb.ai/ to sign up to MindsDB and generate an API key. Once you've done this set the `MINDSDB_API_KEY` environment variable:

In [1]:
import getpass
import os

os.environ["MINDSDB_API_KEY"] = getpass.getpass("Enter your MINDSDB API key: ")

Enter your MINDSDB API key:  ········


If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [2]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

The LangChain AIMind (MindsDB) integration lives in the `langchain-community` package:

In [ ]:
%pip install langchain-community

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [3]:
from langchain_community.chat_models import ChatAIMind

llm = ChatAIMind(
    model="gpt-3.5-turbo",
    temperature=0,
    max_tokens=None,
    # mindsdb_api_key="...",  # if you prefer to pass api key in directly instead of using env vars
    # mindsdb_base_url="...",
    # other params...
)

## Invocation

In [4]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to Spanish. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='Me encanta programar.', response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 31, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4ec11c69-985d-44e1-8b4f-ade22ffdfff0-0')

In [5]:
print(ai_msg.content)

Me encanta programar.


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Italian",
        "input": "I love programming.",
    }
)

AIMessage(content='Adoro programmare.', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 26, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-50c6faf0-47b7-42b4-8a52-4f9c42ab09e4-0')

## Tool calling

AIMind also supports [tool calling](https://platform.openai.com/docs/guides/function-calling) (we use "tool calling" and "function calling" interchangeably here) for several models that lets you describe tools and their arguments, and have the model return a JSON object with a tool to invoke and the inputs to that tool. tool-calling is extremely useful for building tool-using chains and agents, and for getting structured outputs from models more generally.

Given below is a list of models that support tool calling:
- gpt-3.5-turbo
- claude-3-haiku
- firefunction-v1
- hermes-2-pro
- mistral-7b
- mixtral-8x7b
- gemini-1.5-pro

### ChatAIMind.bind_tools()

With `ChatAIMind.bind_tools`, we can easily pass in Pydantic classes, dict schemas, LangChain tools, or even functions as tools to the model. Under the hood these are converted to an OpenAI tool schemas, which looks like:
```
{
    "name": "...",
    "description": "...",
    "parameters": {...}  # JSONSchema
}
```
and passed in every model invocation.

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field


class GetWeather(BaseModel):
    """Get the current weather in a given location"""

    location: str = Field(..., description="The city and state, e.g. San Francisco, CA")


llm_with_tools = llm.bind_tools([GetWeather])

In [8]:
ai_msg = llm_with_tools.invoke(
    "what is the weather like in San Francisco",
)
ai_msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W2Td6ajWeeM3TtRAKuapX6GH', 'function': {'arguments': '{"location":"San Francisco"}', 'name': 'GetWeather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 70, 'total_tokens': 85}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-40bb083c-37fa-450b-90eb-bb6786c73443-0', tool_calls=[{'name': 'GetWeather', 'args': {'location': 'San Francisco'}, 'id': 'call_W2Td6ajWeeM3TtRAKuapX6GH'}])

### AIMessage.tool_calls
Notice that the AIMessage has a `tool_calls` attribute. This contains in a standardized ToolCall format that is model-provider agnostic.

In [9]:
ai_msg.tool_calls

[{'name': 'GetWeather',
  'args': {'location': 'San Francisco'},
  'id': 'call_W2Td6ajWeeM3TtRAKuapX6GH'}]

For more on binding tools and tool call outputs, head to the [tool calling](/docs/how_to/function_calling) docs.